In [6]:
import Pkg

Pkg.add("Interpolations")
Pkg.add("Plots")
Pkg.add("PyPlot")
Pkg.add("PyCall")
Pkg.add("OrdinaryDiffEq")
Pkg.add("YAML")
Pkg.add("DelimitedFiles")
Pkg.add("CSV")
Pkg.add("HDF5")
Pkg.add("StructArrays")
Pkg.add("Random")
Pkg.add("NBInclude")

using Interpolations, Plots, PyCall, OrdinaryDiffEq,
YAML, DelimitedFiles, CSV, HDF5, StructArrays, Random, 
NBInclude, PyPlot

@nbinclude("plumefunctions.ipynb")

ct = pyimport("cantera")

#ENV["PYTHON"] = "/opt/conda/envs/lae2020/bin/python"
#Pkg.build("PyCall")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/

PyObject <module 'cantera' from '/opt/conda/envs/lae2020/lib/python3.8/site-packages/cantera/__init__.py'>

In [9]:
n_species = 53
upper = 20000 #[m]
lower = 16000 #[m]
space = convert(Int, (upper - lower) / 2000 + 1) #250
h = Int.(LinRange(16000, 20000, space))
g = 1

n = 100 #90 #n steps in x dir
s = 50 #80 #s steps in y direction
neg_sum = zeros(s)
pos_sum = zeros(s)
T_save = zeros(length(h), s, n)
u_save = zeros(length(h), s, n)
χ_save = zeros(length(h), s, n, n_species)
x_save = zeros(n)
y_save = zeros(s)
gas_g = StructArray{gas_type}(undef,s,n,length(h))
gas_g .= [gas_type(0)]

for m = 1:length(h)
    h_string = string(h[m])

    ### IMPORT SHOCK EXIT CONDITIONS ###

    u0 = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/u")#initial plume velocity
    u0 = convert(AbstractFloat, u0[2])

    T0 = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/T") #initial plume temperature
    T0 = convert(AbstractFloat, T0[2])

    χ0 = 4.e4 #tracer species
    p_all = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/P")
    p = convert(AbstractFloat, p_all[2])

    u_a = 1.11849E-19 * big(h[m])^5 - 1.14814E-14 * big(h[m])^4 + 4.22542E-10 * big(h[m])^3 - 6.92322E-06 * big(h[m])^2 + 6.58761E-02 * big(h[m]) + 5.37920E+01

    #curve fit #a = ambient vel [m/s] (speed of rocket) 
    T_a = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/T_a")
    χ_a = 70.0 #passive tracer mixing ratio [ppm], just for vincent calculations

    ### CALCULATE VELOCITY AND TEMPERATURE FIELDS (NO CHEMISTRY) ###

    Δϕ = 0.5 * ones(n) #step size in phi
    for i = 2:n
        Δϕ[i] = 2 * Δϕ[i-1] #enlarge with each step by 1.1
    end

    Δψ = 0.5 * ones(s) #s vertical grid points
    for i = 2:s
        Δψ[i] = 1.1 * Δψ[i-1] #enlarge with each step by 1.1
    end

    Pr = 1.0 #prandtl number
    Le = 1.0 #lewis number
    u_init = u0 .* ones(s)
    T_init = T0 .* ones(s)
    χ_init = χ0 .* ones(s)

    R = 287.0
    y_init = compute_y(u_init, T_init, Δψ, R, p)

    #Geometry of plume
    radius = 1.147
    u_init[y_init.>radius] .= convert(AbstractFloat, u_a)
    T_init[y_init.>radius] .= convert(AbstractFloat, T_a)
    χ_init[y_init.>radius] .= convert(AbstractFloat, χ_a)

    ambient = AmbientConditions(u_a, T_a, χ_a, 1.0, 1.0, R, p, T0, u0, χ0)

    #Solve for T and u at all steps: NO CHEMISTRY
    x, y, u, T, χ_vin, ϵ = solve_exhaust_flow(u_init, T_init, ambient, n, Δϕ, Δψ, χ_init=χ_init)
    x_save = x
    y_save = y
    T_save[m, :, :] = T
    u_save[m, :, :] = u

    ### CHEMISTRY STARTS HERE

    #SET INITIAL PLUME CONDITIONS
    #import concentration data from upstream combustion
    χ0_full = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/X")
    χ0 = zeros(n_species)
    #for d = 1: length(χ0_full[:,2])-1
    #    χ0[d] = χ0_full[d,2]

    #end
    χ0[49] = 1000000 #FOR MASS CONS TEST, DELETE LATER 49

    #ppm must sum to a million

    #Redefine species initial conditions for multiple species
    χ_a = zeros(n_species)

    χ_a[48] = 780790 #ppm N2
    χ_a[4] = 209445 #ppm O2
    χ_a[49] = 0 #ppm Ar #CHANGE BACK AFTER DEBUG TO 9339
    χ_a[16] = 9765 #ppm CO2 426

    χ_init = zeros(s, n_species)

    ### COMPUTE MOLE FRACTIONS IN PLUME ###

    i = y_init .>= radius
    for j = 1:length(i) 
        if i[j] == 0 #if y position is less than the initial plume radius, assign plume conditions
            for k = 1:n_species
                χ_init[j, k] = χ0[k] #initial plume conditions
            end
        else #if y position is greater than the initial plume radius, assign ambient conditions
            χ_init[j, 48] = χ_a[48]
            χ_init[j, 4] = χ_a[4]
            χ_init[j, 49] = χ_a[49]
            χ_init[j, 16] = χ_a[16]
        end
    end

    # Save ambient conditions to a struct to call later easily
    ambient = AmbientConditionsχ(u_a, T_a, χ_a, 1.0, 1.0, R, p, T0, u0, χ0)

    # Create 3D array to save concentrations at each y,x location
    χ = zeros(length(χ_vin[:, 1]), length(χ_vin[1, :]), n_species) #[y,x,species]

    χ[:, 1, :] = χ_init # at x=1 assign concentrations for all species at all y

    # Initialize arrays for Strang-Splitting
    χ_h0 = zeros(size(χ_init))
    χ_1 = zeros(size(χ_init))
    
    # Create gas object to store Reactor output gas object state
    gas = ct.Solution("gri30.yaml")

    # Create a dummy reactor to ????
    dummy_reactor = ct.IdealGasReactor(gas)

    for i = 1:n-1 #x
        
        for j = 1:n_species #species
            #calculate f0 at half step 0.5*Δϕ (x)
            χ_h0[:, j] = solve_exhaust_flow_χ(u[:, i], T[:, i], ambient, n, 0.5 * Δϕ[i], Δψ, χ[:, i, j], i, j)
            # concentration of species j at x = i and y = all
        end

        save_tuple = solve_reaction(χ_h0, T[:, i], Δϕ[i], ϵ[i], u[:, i], gas, i, χ_1, s, n_species, @view gas_g[:,i,m])
        gas_g.gas[:,i,m] .= save_tuple[2]
        χ_1 = save_tuple[1]

        for j = 1:n_species-1 #species
            #calculate f0 at full step Δϕ
            χ[:, i+1, j] = solve_exhaust_flow_χ(u[:, i], T[:, i], ambient, n, 0.5 * Δϕ[i], Δψ, χ_1[:, j], i, j)
        end

        i = i + 1
    end
    χ_save[m, :, :, :] = χ

    print(m, " of ", length(h), " altitudes done!\n")
end
println("done computing! 💕")

1 of 3 altitudes done!
2 of 3 altitudes done!
3 of 3 altitudes done!
done computing! 💕


In [ ]:
# Define altitude range to plot over
h = Int.(LinRange(16000, 20000, space)) #make an array and put in loop for all alts
h_string_arr = string(h)

for m = 1:length(h) #For all specified altitudes

    # Initialize arrays to save results
    T = T_save[m, :, :] #temperature
    u = u_save[m, :, :] #velocity
    χ = χ_save[m, :, :, :] #concentrations
    x = x_save
    y = y_save
    h_string = string(h[m])
    P_atm = 101325 #placeholder, need altitude dependent
    R = 8.314 #placeholder

    #REGRID SOLUTION
    #xx, yy, u_g, T_g, χ_gH2O =  regrid_solution(x, y, u, T, χ[:,:,6], 0.01);
    #xx, yy, u_g, T_g, χ_gH2 =  regrid_solution(x, y, u, T, χ[:,:,1], 0.01);
    xx, yy, u_g, T_g, χ_gO2 =  regrid_solution(x, y, u, T, χ[:,:,4], 0.01)
    xx, yy, u_g, T_g, χ_gN2 =  regrid_solution(x, y, u, T, χ[:,:,48], 0.01)
    xx, yy, u_g, T_g, χ_gNO2 = regrid_solution(x, y, u, T, χ[:, :, 37], 0.01)
    xx, yy, u_g, T_g, χ_gN2O = regrid_solution(x, y, u, T, χ[:, :, 38], 0.01)
    xx, yy, u_g, T_g, χ_gNO = regrid_solution(x, y, u, T, χ[:, :, 36], 0.01)
    xx, yy, u_g, T_g, χ_gAr = regrid_solution(x, y, u, T, χ[:, :, 49], 0.01)

    #PLOT 2D MAPS AND SAVE
    x_max = 20 #[m]
    #plot_H2O = plot_heatmap(xx, yy, χ_gH2O, "Axial distance, m", "Radial distance, m","Concentration", :magma, x_max)
    #savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/X_H2O.png")
    #plot_NO2 = plot_heatmap(xx, yy, χ_gNO2, "Axial distance, m", "Radial distance, m","Concentration", :magma, x_max)
    #savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/X_NO2.png")
    #plot_N2O = plot_heatmap(xx, yy, χ_gN2O, "Axial distance, m", "Radial distance, m","Concentration", :magma, x_max)
    #savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/X_N2O.png")
    #plot_NO = plot_heatmap(xx, yy, χ_gNO, "Axial distance, m", "Radial distance, m","Concentration", :magma, x_max)
    #savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/X_NO.png")
    plot_Ar = plot_heatmap(xx, yy, χ_gAr, "Axial distance, m", "Radial distance, m", "Concentration", :magma, x_max)
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/X_Ar.png")
    plot_N2 = plot_heatmap(xx, yy, χ_gN2, "Axial distance, m", "Radial distance, m", "Concentration", :magma, x_max)
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/X_N2.png")
    plot_O2 = plot_heatmap(xx, yy, χ_gO2, "Axial distance, m", "Radial distance, m", "Concentration", :magma, x_max)
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/X_O2.png")

    #CALCULATE NO EI FOR ALTITUDE
    # Sum of all NOx species as a function of x
    sumNO = zeros(n)
    sumNO2 = zeros(n)
    sumN2O = zeros(n)
    Xarea = zero(u_g)
    EI_Ar = zero(u_g)
    i = 1
    j = 1

    x_lim = length(xx)
    y_lim = length(yy)

    MW_Ar = 39.95 #[kg/mol]
    mdot_fuel = 67.35 #[kg/s]
    rho_tot = zeros(s,n)
    MFar = zeros(s,n)
    MFn2 = zeros(s,n)
    MFo2 = zeros(s,n)
    mdot_Ar = zeros(s,n)
    mdot_N2 = zeros(s,n)
    mdot_O2 = zeros(s,n)

    Δψ = 0.5 * ones(s) #s vertical grid points
    for i = 2:s
        Δψ[i] = 1.1 * Δψ[i-1] #enlarge with each step by 1.1
    end

    Δϕ = 0.5 * ones(n) #step size in phi
    for i = 2:n
        Δϕ[i] = 2 * Δϕ[i-1] #enlarge with each step by 1.1
    end

    for i = 1:n-1, j = 1:s-1
        rho_tot[j,i] = gas_g.gas[j,i,m].density_mass[1] #[kg/m^3] #need to convert to xx yy coordinates
        MFar[j,i] = gas_g.gas[j,i,m]("Ar").Y[1] #[kg/kg tot]
        MFn2[j,i] = gas_g.gas[j,i,m]("N2").Y[1] #[kg/kg tot]
        MFo2[j,i] = gas_g.gas[j,i,m]("O2").Y[1] #[kg/kg tot]
        Xarea[j,i] = 3.1415*(Δψ[j+1]^2 - Δψ[j]^2)
        mdot_Ar[j,i] = Xarea[j,i]*MFar[j,i]*rho_tot[j,i]*u[j,i]
        mdot_N2[j,i] = Xarea[j,i]*MFn2[j,i]*rho_tot[j,i]*u[j,i]
        mdot_O2[j,i] = Xarea[j,i]*MFo2[j,i]*rho_tot[j,i]*u[j,i]
    end

    xx, yy, u_g, T_g, rho_tot_g = regrid_solution(x, y, u, T, rho_tot, 0.01)
    xx, yy, u_g, T_g, MFar_g = regrid_solution(x, y, u, T, MFar, 0.01)
    xx, yy, u_g, T_g, MFn2_g = regrid_solution(x, y, u, T, MFn2, 0.01)
    xx, yy, u_g, T_g, MFo2_g = regrid_solution(x, y, u, T, MFo2, 0.01)
    xx, yy, u_g, T_g, mdot_Ar_g = regrid_solution(x, y, u, T, mdot_Ar, 0.01)
    xx, yy, u_g, T_g, mdot_N2_g = regrid_solution(x, y, u, T, mdot_N2, 0.01)
    xx, yy, u_g, T_g, mdot_O2_g = regrid_solution(x, y, u, T, mdot_O2, 0.01)

    flux_N2 = zeros(x_lim)
    flux_Ar = zeros(x_lim)
    flux_O2 = zeros(x_lim)

    for i = 1:x_lim-1, j = 1:y_lim-1
        flux_N2[i] += mdot_N2_g[j,i]
        flux_Ar[i] += mdot_Ar_g[j,i]
        flux_O2[i] += mdot_O2_g[j,i]
    end
    #print(mdot_Ar_g[:,1])
    #itp = interpolate((xx,), flux_Ar, Gridded(Linear()))
    #
    #plot(xx, itp)
    #savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/" * h_string * "m/flux_Ar.png")

    plot(xx[1:13], mdot_Ar_g[1,1:13], xlabel = "Axial Distance [m]", ylabel = "mdot Ar in x dir at y = 0")
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/" * h_string * "m/mdot_Ar_y0.png")
    
    plot(xx[1:13], flux_N2[1:13])
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/" * h_string * "m/flux_N2.png")

    plot(xx[1:13], flux_O2[1:13])
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/" * h_string * "m/flux_O2.png")

    EI_Ar = Xarea.*MFar_g.*rho_tot_g.*u_g./mdot_fuel
    
    plot_heatmap(xx, yy, mdot_Ar_g, "Axial distance, phi", "Radial distance, psi", "mdot Ar", :magma, x_max)
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/" * h_string * "m/mdot_Ar.png")

    plot_heatmap(xx, yy, u_g, "Axial distance, m", "Radial distance, m", "vel", :magma, x_max)
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/" * h_string * "m/u_g.png")

    plot_heatmap(xx, yy, EI_Ar, "Axial distance, m", "Radial distance, m", "EI Ar", :magma, x_max)
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/" * h_string * "m/EI_Ar.png")

    plot_heatmap(xx, yy, mdot_N2_g, "Axial distance", "Radial distance", "mdot N2", :magma, x_max)
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/" * h_string * "m/mdot_N2.png")

    print("plotted ", h_string[m], " altitude!\n")
end

print("\n done plotting!")

In [ ]:
#calculate how much mass diffuses into next cell (axial direction)
#sum should equal initial amount (kg diffused per Δx)